<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_submitJob_DSapp_OpenSees_Detailed.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Run OpenSees -- Detailed 📒
***Submit an OpenSees App***

by Silvia Mazzoni, DesignSafe, 2025

This notebook serves as a template for submitting the following DesignSafe OpenSees Apps:
* openSees-express
* openSees-mp-s3
* opensees-sp-s3

**We are using previously-defined python function to streamline the process.**

In [1]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


## Connect to Tapis

In [2]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-23T01:42:21+00:00
 Token expires in: 2:37:28.196521
-- LOG IN SUCCESSFUL! --


In [3]:
print('QUEUE INFO')
OpsUtils.get_system_queues(t,system_id="stampede3",display=True);

QUEUE INFO


name,icx,skx,skx-dev,pvc,spr,nvdimm,h100
description,"icx (CPU, standard compute)","skx (CPU, recommended standard compute)","skx-dev (CPU, 2 hour max, 1 job max, for testing)","pvc (GPU, Intel no CUDA)","spr (CPU, high memory bandwidth)","nvdimm (CPU, large memory)",None
hpcQueueName,icx,skx,skx-dev,pvc,spr,nvdimm,h100
maxJobs,-1,-1,-1,-1,-1,-1,-1
maxJobsPerUser,20,60,3,4,36,3,4
minNodeCount,1,1,1,1,1,1,1
maxNodeCount,32,256,16,4,32,1,4
minCoresPerNode,1,1,1,1,1,1,1
maxCoresPerNode,80,48,48,96,112,80,96
minMemoryMB,1,1,1,1,1,1,1
maxMemoryMB,256000,192000,192000,128000,128000,4000000,1000000


---
## App User Input

### Initialize

In [4]:
# initalize
tapisInput = {}
tapisInput["name"] = 'OpsTrain_JobSubmit_OpenSees'

### App Parameters

In [5]:
tapisInput["appId"] = "opensees-mp-s3" # options: "opensees-express", "opensees-mp-s3", "opensees-2p-s3"
tapisInput["appVersion"] = "latest" # always use latest in this Notebook Template

### TACC-Job Parameters
https://docs.tacc.utexas.edu/hpc/stampede3/

In [6]:
tapisInput["maxMinutes"] = 6

# OpenSees-mp-s3 and OpenSees-xp-s3 only:
if tapisInput["appId"] in ["opensees-mp-s3","opensees-sp-s3"]:
    tapisInput["execSystemId"] = "stampede3" # the app runs on stampede only
    tapisInput["execSystemLogicalQueue"] = "skx-dev" # "skx", "skx-dev"... for info: use the command 'OpsUtils.get_system_queues(t,system_id="stampede3",display=True);'
    tapisInput["nodeCount"] = 1 # limits set by which compute nodes you use
    tapisInput["coresPerNode"] = 48 # limits set by which compute nodes you use
    tapisInput["allocation"] = "DS-HPC1"


---
###  INPUT Files Parameters

#### Storage SystemTapis & Tapis Base Path in URI format
this is the very first part of your path, just above your home folder.

Options: 
* **CommunityData**
* **Published**

The following options are user or project-dependent, and require unique path input.


The following option requires additional **user-dependent** input:
* **MyData**

The following option requires additional **user- and system- dependent** input:
* **Work**

The following option requires additional **project-dependent** input:
* **MyProjects**

You can obtain a dependent tapis-URI path by performing the first step of submitting an OpenSeesMP job at the app portal: https://www.designsafe-ci.org/workspace/opensees-mp-s3

In [7]:
tapisInput['storage_system'] = 'Work/stampede3' # options: Community,MyData,Published,MyProjects,Work/stampede3,Work/frontera,Work/ls6

### We will use a utility function to get the Tapis-compatible URI:

In [8]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils,['get_user_path_tapis_uri.py'])

In [9]:
tapisInput['storage_system_baseURL'] = OpsUtils.get_user_path_tapis_uri(t,tapisInput['storage_system'])

print('storage_system_baseURL:',tapisInput['storage_system_baseURL'])

found paths file: /home/jupyter/MyData/.tapis_user_paths.json
storage_system_baseURL tapis://designsafe.storage.default/silvia/


#### If your input files are in **user-specific** (*Work*) or **project-specific** (*MyProjects*) storage systems 

In [10]:
# # Work is user- and system-dependent
# if tapisInput['storage_system']=='Work':
#     tapisInput['storage_system_baseURL'] = 'tapis://cloud.data/work/05072/silvia/stampede3'

# # The following are project-dependent
# #  This value needs to be updated for each project
# if tapisInput['storage_system']=='MyProjects':
#     tapisInput['storage_system_baseURL'] = 'tapis://project-7997906542076432871-242ac11c-0001-012'

In [11]:
# tapis_uri = OpsUtils.get_user_path_tapis_uri(t,tapisInput['storage_system'])
# print(tapis_uri)

#### File Paths
The **input_folder**  is the directory of your input file.
* **DO NOT INCLUDE** the storage system, such as MyData, etc. 
* Start from the first folder within your storage-system folder.

The **Main Script** is the full name of the file that will be submitted to OpenSees. Yes you need to include the extension.

In [ ]:
tapisInput['input_folder'] = '_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'
tapisInput['Main Script'] = 'Ex1a_verymany.Canti2D.Push.mp.tcl'

---
### OUTPUT-Files Parameters
Where would you like your files to be archived once the job is finished?

Options: **MyData** and **Work** 

In both cases you will find them in the **tapis-jobs-archive** folder in either MyData or Work/stampede3. 

Remember, you cannot write to Projects nor CommunityData.

In [ ]:
tapisInput['archive_system']='Work' # Options: MyData or Work

---
### Review User Input

In [ ]:
display(tapisInput)

#### Get interpreted Tapis-App Input (Optional)
this is what will be sent as input (just informational here, and good to check input)

In [ ]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils,['get_tapis_job_description.py'])

In [ ]:
OpsUtils.get_tapis_job_description(t,tapisInput)

### GO!

In [ ]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils,['run_tapis_job.py'])

In [ ]:
# We have created a function for each task and then combined them into a single function
jobReturns = OpsUtils.run_tapis_job(t,tapisInput)

In [ ]:
jobUuid = jobReturns['jobUuid']
print('jobUuid:',jobUuid)
print('jobReturns',jobReturns.keys())

<hr>

### ONCE THE JOB HAS COMPLETED....

<hr>

## Get detailed Job Status, Metadata, History, Stage Durations, and Files List

In [ ]:
JobStatus = OpsUtils.get_tapis_job_status(t, jobUuid)

In [ ]:
JobMetadata = OpsUtils.get_tapis_job_metadata(t, jobUuid)

In [ ]:
JobHistory = OpsUtils.get_tapis_job_history_data(t, jobUuid,print_out=True)

In [ ]:
AllFilesDict = OpsUtils.get_tapis_job_all_files(t, jobUuid, displayIt=10, target_dir=False)

## Visualize Data
this is the same process as what we had done when we presented the web-portal submit

---
#### get base path for output data from posted path:
Different systems in DesignSafe have different root paths

In [ ]:
basePath = JobMetadata['archiveSystemDir_out']
print('basePath',basePath)

#### directory contents

In [ ]:
if os.path.exists(basePath):
    print(os.listdir(basePath))
else:
    print('path does not exist')

---
#### Plot some analysis results
for any of the above analyses

In [ ]:
import matplotlib.pyplot as plt
import numpy

In [ ]:
#pick any case
print('basePath:',basePath)
dataDir = f'{basePath}/DataTCLmp'; # know this from my input script, or see directory contents
print('dataDir:',dataDir)
if os.path.exists(dataDir):
    print('dataDir exists!!!')
else:
    print('dataDir DOES NOT EXIST! -- it may just need time....you may just need to re-run this and the subsequent cells')

#### List files in folder for a specific case, also using a wildcard
You could use the following command, but it is not 100% reliable nor safe. it also doesn't return the list.
*os.system(f'ls {dataDir}/*Lcol{Lcol}.out')*

In [ ]:
Lcol = 100.0

In [ ]:
import glob
# Build the wildcard pattern
pattern = os.path.join(dataDir, f"*Lcol{Lcol}.out")
# Get matching files as a Python list
files = glob.glob(pattern)
for f in files:
    print(f)

In [ ]:
plt.close('all')
fname3o = f'DFree_Lcol{Lcol}.out'
fname3 = f'{dataDir}/{fname3o}'
print('fname3:',fname3)
dataDFree = numpy.loadtxt(fname3)
plt.subplot(211)
plt.title(f'Ex1a.Canti2D Lcol={Lcol}')
plt.grid(True)
plt.plot(dataDFree[:,1])
plt.xlabel('Step Number')
plt.ylabel('Free-Node Displacement')
plt.subplot(212)
plt.grid(True)
plt.plot(dataDFree[:,1],dataDFree[:,0])
plt.xlabel('Free-Node Disp.')
plt.ylabel('Pseudo-Time (~Force)')
plt.savefig(f'{dataDir}/Response.jpg')
plt.show()
print(f'plot saved to {dataDir}/Response_Lcol{Lcol}.jpg')
print('End of Run: Ex1a.Canti2D.Push.py.ipynb')


In [ ]:
print('Done!')